# Capstone Job Application Assistant — Lightweight Notebook

This lightweight notebook performs resume extraction, keyword-based skill extraction,
resume↔JD matching, templated cover letter generation, and saves application records.

It uses your uploaded resume at `/mnt/data/resume.pdf` (already uploaded). Run cells in order.

---

In [ ]:
!pip install -q pdfplumber
print('pdfplumber installed (or already present)')

In [ ]:
import os, json, textwrap
from pathlib import Path
import pdfplumber
from datetime import datetime

# Paths
RESUME_PATH = '/mnt/data/resume.pdf'  # uploaded resume file
MEMORY_PATH = '/content/job_app_memory.json'

# Helpers

def extract_text_from_pdf(path):
    text = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text() or ""
            text.append(page_text)
    return "\n\n".join(text).strip()


def simple_skill_extract(text):
    keywords = [
        'python','java','spring','mysql','html','css','javascript',
        'devops','docker','git','linux','aws','azure','react','node','mongodb'
    ]
    low = text.lower()
    return [k for k in keywords if k in low]


def simple_match(resume_text, jd_text):
    resume_skills = set(simple_skill_extract(resume_text))
    jd_skills = set(simple_skill_extract(jd_text))
    matched = sorted(list(resume_skills & jd_skills))
    missing = sorted(list(jd_skills - resume_skills))
    score = round(len(matched) / max(1, len(jd_skills)) * 100.0, 2) if jd_skills else 0.0
    return {
        'similarity_score': score,
        'matched_skills': matched,
        'missing_skills': missing,
        'resume_skills': sorted(list(resume_skills)),
        'jd_skills': sorted(list(jd_skills))
    }


def generate_cover_letter(resume_text, jd_text, name='Candidate', role='the role'):
    resume_intro = (resume_text.splitlines()[0] if resume_text.splitlines() else '')[:200]
    jd_intro = (jd_text.splitlines()[0] if jd_text.splitlines() else '')[:200]
    skills = ', '.join(simple_skill_extract(resume_text)) or 'programming and web development'
    intro = f"Dear Hiring Team,\n\nI am writing to express my interest in {role}. I am {name}."
    body = (
        f"\n\nFrom my resume: {resume_intro}\n\nFrom the JD: {jd_intro}\n\nRelevant skills: {skills}."
    )
    closing = f"\n\nThank you.\n{name}"
    return intro + body + closing


In [ ]:
# Load resume text and preview
if not os.path.exists(RESUME_PATH):
    print(f"Resume not found at {RESUME_PATH}. Upload to Colab or change RESUME_PATH.")
else:
    resume_text = extract_text_from_pdf(RESUME_PATH)
    print('Resume loaded. Preview (first 800 chars):\n')
    print(resume_text[:800])
    print('\nDetected skills from resume:', simple_skill_extract(resume_text))


In [ ]:
# Example JD and one-off run
sample_jd = """We are hiring a Software Engineer experienced in Java, Spring Boot, MySQL, HTML/CSS, and Git. Familiarity with Linux and Docker is a plus."""

result = simple_match(resume_text, sample_jd)
print('Match result:', result)
print('\nGenerated cover letter (preview):\n')
print(generate_cover_letter(resume_text, sample_jd, name='Gideon Kingsly Raj', role='Software Engineer'))


In [ ]:
# Function to analyze a JD and optionally save the application to memory

def analyze_jd_and_save(jd_text, name='Gideon Kingsly Raj', role='Software Engineer', save=True):
    result = simple_match(resume_text, jd_text)
    cover = generate_cover_letter(resume_text, jd_text, name=name, role=role)
    result['cover_letter'] = cover
    result['timestamp'] = datetime.utcnow().isoformat() + 'Z'
    if save:
        mem = []
        if os.path.exists(MEMORY_PATH):
            try:
                mem = json.load(open(MEMORY_PATH,'r',encoding='utf8'))
            except Exception:
                mem = []
        mem.append({'jd': jd_text, 'result': result})
        json.dump(mem, open(MEMORY_PATH,'w',encoding='utf8'), indent=2)
        print(f'Saved to {MEMORY_PATH}')
    print('\n=== Analysis ===')
    print(json.dumps(result, indent=2))
    return result

# Quick test example
if __name__ == '__main__':
    test_jd = 'We require a developer with Python, Docker, Git, and AWS experience.'
    analyze_jd_and_save(test_jd)


## How to use

1. Upload this notebook to Google Colab.
2. Run the cells in order.
3. If your resume is not at `/mnt/data/resume.pdf`, upload it in Colab and update `RESUME_PATH`.
4. Replace `sample_jd` or call `analyze_jd_and_save(jd_text, name, role, save=True)` interactively to analyze and save applications.

The saved application records will be stored in `/content/job_app_memory.json`.